In [6]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import pandas as pd
import numpy as np

df_pesoEaltura = pd.read_csv("/content/drive/MyDrive/Data/BDIPMamaV11-INTELIPesoEAltura_DATA_LABELS_2023-01-24_1926.csv")

In [20]:
df_pesoEaltura = df_pesoEaltura.rename(columns={"Record ID": "record_id",
                                                "Data: ": "data",
                                                "Peso": "peso",
                                                "Altura (em centimetros)": "altura"})

lista_RecorId = df_pesoEaltura['record_id'].unique()
df_novo = pd.DataFrame()

for i in lista_RecorId: 
    df_query = df_pesoEaltura.query(f'record_id == {i}')
    df_query = df_query.sort_values(by='data')

    df_row_first = df_query.iloc[0]

    if len(df_query) > 1: 
      df_row_second = df_query.iloc[len(df_query)-1]

      numero = len(df_query) - 2

      for col in ['altura', 'peso', 'IMC']:
        while (pd.isna(df_row_second['data'])) and (numero > 1):
          df_row_second = df_query.iloc[len(df_query) - numero - 1]
          numero -= 1

      df_row_first['ultimo_peso'] = df_row_second['peso']
      df_row_first['ultima_altura'] = df_row_second['altura']
      df_row_first['ultimo_IMC'] = df_row_second['IMC']
      df_row_first['ultima_data'] = df_row_second['data']
      
      df_novo = df_novo.append(df_row_first, ignore_index=True)
      
    else: 
      df_novo = df_novo.append(df_row_first, ignore_index=True)

df_novo = df_novo.rename(columns={
                        "peso": "primeiro_peso", 
                        "altura": "primeira_altura",
                        "IMC": "primeiro_IMC"
                        })

df_pesoEaltura = df_novo

/usr/local/lib/python3.8/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1724: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer_missing(indexer, value)


In [21]:
np.random.seed(73)

df_pesoEaltura['primeiro_IMC'] = (df_pesoEaltura['primeiro_peso'] / pow(df_pesoEaltura['primeira_altura']/100, 2))

lista_pesoEaltura = ["primeiro_peso", "primeira_altura", "ultimo_peso", "ultima_altura", "primeiro_IMC"]
lista_normal = []

for i in lista_pesoEaltura: 
  df_pesoEaltura[i] = df_pesoEaltura[i].fillna(pd.Series(np.random.normal(df_pesoEaltura[i].std(skipna=True), 
                                                                          df_pesoEaltura[i].std(skipna=True), 
                                                                          df_pesoEaltura[i].isnull().sum()), 
                                                         index=df_pesoEaltura[i].index[df_pesoEaltura[i].isnull()]))

df_pesoEaltura['ultimo_IMC'] = (df_pesoEaltura['ultimo_peso'] / pow(df_pesoEaltura['ultima_altura']/100, 2))

df_pesoEaltura

,record_id,Repeat Instrument,Repeat Instance,data,primeiro_peso,primeira_altura,primeiro_IMC,ultimo_peso,ultima_altura,ultimo_IMC,ultima_data
0,302.0,Dados Antropometricos,20.0,2009-01-02,57.80,158.0,23.153341,57.000000,158.000000,22.832879,2009-01-23
1,710.0,Dados Antropometricos,18.0,2008-07-12,61.00,156.0,25.065746,60.500000,155.000000,25.182102,2008-10-01
2,752.0,Dados Antropometricos,2.0,2009-12-03,84.00,152.0,36.357341,82.500000,152.000000,35.708102,2010-07-27
3,1367.0,Dados Antropometricos,1.0,2010-04-28,57.90,143.0,28.314343,58.000000,143.000000,28.363245,2010-06-28
4,1589.0,Dados Antropometricos,12.0,2009-04-03,63.20,167.0,22.661264,63.300000,167.000000,22.697121,2009-04-24
...,...,...,...,...,...,...,...,...,...,...,...
4267,82123.0,Dados Antropometricos,5.0,2021-02-07,85.40,153.0,36.481695,84.100000,153.000000,35.926353,2021-02-28
4268,82124.0,Dados Antropometricos,1.0,2020-11-10,81.10,151.0,35.568615,29.283682,9.236831,3432.256009,NaN
4269,82131.0,Dados Antropometricos,1.0,2020-11-01,71.40,156.5,29.152079,73.350000,156.000000,30.140533,2020-11-24
4270,82205.0,Dados Antropometricos,4.0,2021-02-28,135.85,174.0,44.870525,134.500000,178.000000,42.450448,2021-04-28


In [22]:
df_pesoEaltura['diferenca_peso'] = df_pesoEaltura['ultimo_peso'] - df_pesoEaltura['primeiro_peso']
df_pesoEaltura['coef_peso'] = df_pesoEaltura['ultimo_peso'] / df_pesoEaltura['primeiro_peso']

df_pesoEaltura['ultima_data'] = pd.to_datetime(df_pesoEaltura['ultima_data'])
df_pesoEaltura['data'] = pd.to_datetime(df_pesoEaltura['data'])

df_pesoEaltura['diferenca_altura'] = df_pesoEaltura['ultima_data'] - df_pesoEaltura['data']

df_pesoEaltura['diferenca_tempo'] = (df_pesoEaltura['ultima_data'] - df_pesoEaltura['data']).dt.days
df_pesoEaltura['coef_peso_tempo'] = df_pesoEaltura['diferenca_peso'] / df_pesoEaltura['diferenca_tempo']

df_pesoEaltura['Repeat Instrument'] = df_pesoEaltura['Repeat Instrument'].fillna(0)
df_pesoEaltura['Repeat Instance'] = df_pesoEaltura['Repeat Instance'].fillna(0)
df_pesoEaltura['data'] = df_pesoEaltura['data'].fillna(0)
df_pesoEaltura['ultima_data'] = df_pesoEaltura['ultima_data'].fillna(0)
df_pesoEaltura['diferenca_altura'] = df_pesoEaltura['diferenca_altura'].fillna(pd.Timedelta(0))

df_pesoEaltura['diferenca_tempo'] = df_pesoEaltura['diferenca_tempo'].fillna(0)
df_pesoEaltura['coef_peso_tempo'] = df_pesoEaltura['coef_peso_tempo'].fillna(0)

In [23]:
df_pesoEaltura

,record_id,Repeat Instrument,Repeat Instance,data,primeiro_peso,primeira_altura,primeiro_IMC,ultimo_peso,ultima_altura,ultimo_IMC,ultima_data,diferenca_peso,coef_peso,diferenca_altura,diferenca_tempo,coef_peso_tempo
0,302.0,Dados Antropometricos,20.0,2009-01-02 00:00:00,57.80,158.0,23.153341,57.000000,158.000000,22.832879,2009-01-23 00:00:00,-0.800000,0.986159,21 days,21.0,-0.038095
1,710.0,Dados Antropometricos,18.0,2008-07-12 00:00:00,61.00,156.0,25.065746,60.500000,155.000000,25.182102,2008-10-01 00:00:00,-0.500000,0.991803,81 days,81.0,-0.006173
2,752.0,Dados Antropometricos,2.0,2009-12-03 00:00:00,84.00,152.0,36.357341,82.500000,152.000000,35.708102,2010-07-27 00:00:00,-1.500000,0.982143,236 days,236.0,-0.006356
3,1367.0,Dados Antropometricos,1.0,2010-04-28 00:00:00,57.90,143.0,28.314343,58.000000,143.000000,28.363245,2010-06-28 00:00:00,0.100000,1.001727,61 days,61.0,0.001639
4,1589.0,Dados Antropometricos,12.0,2009-04-03 00:00:00,63.20,167.0,22.661264,63.300000,167.000000,22.697121,2009-04-24 00:00:00,0.100000,1.001582,21 days,21.0,0.004762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4267,82123.0,Dados Antropometricos,5.0,2021-02-07 00:00:00,85.40,153.0,36.481695,84.100000,153.000000,35.926353,2021-02-28 00:00:00,-1.300000,0.984778,21 days,21.0,-0.061905
4268,82124.0,Dados Antropometricos,1.0,2020-11-10 00:00:00,81.10,151.0,35.568615,29.283682,9.236831,3432.256009,0,-51.816318,0.361081,0 days,0.0,0.000000
4269,82131.0,Dados Antropometricos,1.0,2020-11-01 00:00:00,71.40,156.5,29.152079,73.350000,156.000000,30.140533,2020-11-24 00:00:00,1.950000,1.027311,23 days,23.0,0.084783
4270,82205.0,Dados Antropometricos,4.0,2021-02-28 00:00:00,135.85,174.0,44.870525,134.500000,178.000000,42.450448,2021-04-28 00:00:00,-1.350000,0.990063,59 days,59.0,-0.022881
